## Représentation sous forme de diagramme de Vornoi des zones nécessitant un aménagement d'arceaux à vélo sur Brest

### Importation des librairies

In [90]:
# Import necessary packages
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import scipy.spatial as spatial  # For spatial operations
import folium # Plot the points on the map as red circles
import os # For file operations

### I. Chargement des données

La cellule suivante permet de récupérer les données contenues dans les fichiers CSV qui ont été préalablement compilés.

Par la suite elle les stocke dans deux variables de types DataFrame afin de pouvoir les réutiliser par la suite.

Enfin elle définit aussi des variables concernant le périmètre ainsi que l'épicentre de la carte qui sera affichée par la suite.

In [103]:
# Get the absolute path to this working directory
cheminAbs = os.getcwd().split(os.path.sep)

# Change the path to the directory CSV
cheminAbs[len(cheminAbs)-1] = "CSV"
chemin = os.path.sep.join(cheminAbs)

# Load the data
df_incidents = pd.read_csv(os.path.join(chemin, "df_incidents.csv"))
df_coords = pd.read_csv(os.path.join(chemin, "df_poi.csv"))
df = df_incidents[df_incidents['categorie'] == 5]

# Define the bounding box based on the center of incidents
lat = df_incidents['latitude'].mean()
lon = df_incidents['longitude'].mean()
delta = 0.05  # Smaller delta for better zooming
box = (lat - delta, lon - delta, lat + delta, lon + delta)

# Define the center location of the bounding box
c1 = (box[0] + box[2] ) / 2
c2 = (box[1] + box[3]) / 2
center_coords = [c1, c2]

### II. Definition de la fonction permettant d'afficher un diagramme de voronoï

Cette cellule de code fournie définit une fonction appelée voronoi_finite_polygons_2d qui prend en entrée un objet Voronoi calculé à l'aide de la bibliothèque scipy. 

Cette fonction reconstruit les régions de Voronoi infinies dans un diagramme 2D en régions finies. Elle renvoie une liste de polygones finis représentant les régions de Voronoi et un tableau numpy contenant les coordonnées des nouveaux sommets ajoutés lors de la reconstruction.

In [92]:
def voronoi_finite_polygons_2d(vor, radius=None):
    """Reconstruct infinite Voronoi regions in a
    2D diagram to finite regions.
    
    Parameters:
    - vor: scipy.spatial.Voronoi object
           Voronoi diagram computed using the scipy library.
    - radius: float, optional
              Radius of the bounding circle around the original points.
              If not provided, it is set to the maximum point-to-point
              distance in the Voronoi diagram.
              
    Returns:
    - new_regions: list
                   List of finite polygons representing Voronoi regions.
    - new_vertices: numpy array
                    Array containing the coordinates of the new vertices
                    added during reconstruction.
    
    Source:
    [https://stackoverflow.com/a/20678647/1595060](https://stackoverflow.com/a/20678647/1595060)
    """
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")
    
    new_regions = []  # List to store finite polygons
    new_vertices = vor.vertices.tolist()  # List to store new vertices
    
    center = vor.points.mean(axis=0)  # Compute the center of the points
    
    # If radius is not provided, set it to the maximum point-to-point distance
    if radius is None:
        radius = vor.points.ptp().max()
    
    # Construct a map containing all ridges for a given point
    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))
    
    # Reconstruct infinite regions
    for p1, region in enumerate(vor.point_region):
        vertices = vor.regions[region]
        
        if all(v >= 0 for v in vertices):
            # Finite region: already in the new_regions list
            new_regions.append(vertices)
            continue
        
        # Reconstruct a non-finite region
        ridges = all_ridges[p1]
        new_region = [v for v in vertices if v >= 0]
        
        for p2, v1, v2 in ridges:
            if v2 < 0:
                v1, v2 = v2, v1
            if v1 >= 0:
                # Finite ridge: already in the region
                continue
            
            # Compute the missing endpoint of an infinite ridge
            t = vor.points[p2] - vor.points[p1]  # Tangent
            t /= np.linalg.norm(t)
            n = np.array([-t[1], t[0]])  # Normal
            midpoint = vor.points[[p1, p2]].mean(axis=0)
            direction = np.sign(np.dot(midpoint - center, n)) * n
            far_point = vor.vertices[v2] + direction * radius
            new_region.append(len(new_vertices))
            new_vertices.append(far_point.tolist())
        
        # Sort region counterclockwise
        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = np.array(new_region)[np.argsort(angles)]
        new_regions.append(new_region.tolist())
    
    return new_regions, np.asarray(new_vertices)

### III. Traitement des données

La cellule de code suivante initialise une liste vide appelée "coos" pour stocker des tuples de coordonnées. 

Ensuite, elle itère sur les lignes de deux DataFrames, extrait les valeurs de latitude et de longitude, les convertit en nombres décimaux, et ajoute les tuples de coordonnées à la liste "coos".

In [93]:
# Initialize an empty list to store coordinate tuples
coos = []

# Iterate over the rows of the DataFrame to extract coordinates
for dataFrame in [df_coords, df]:
    for i in range(len(dataFrame.latitude)):
        # Extract latitude and longitude values from the DataFrame
        x = float(dataFrame.latitude.values[i])
        y = float(dataFrame.longitude.values[i])

        # Append the coordinate tuple (latitude, longitude) to the list
        coos.append((x, y))

In [94]:
# Compute the Voronoi diagram for the given coordinates
vor = spatial.Voronoi(coos)

# Reconstruct finite polygons from infinite Voronoi regions
regions, vertices = voronoi_finite_polygons_2d(vor)

### IV. Affichage d'une carte avec les points d'intérêts

Cette cellule crée une carte interactive en utilisant la bibliothèque Folium en Python. 

Elle centre la carte sur les coordonnées de Brest, puis ajoute des régions de Voronoi en bleu et des marqueurs de cercle rouge et vert pour représenter des points individuels. Enfin, elle affiche la carte.

In [104]:
# Créer une carte centrée sur Brest
m = folium.Map(location=[center_coords[0], center_coords[1]], zoom_start=15)

# Add the Voronoi regions to the map
for region in regions:
    polygon = folium.Polygon(locations=vertices[region], color='blue', fill=True, fill_color='blue', fill_opacity=0.1)
    m.add_child(polygon)

# Ajouter les points individuels à la carte
for i in range(len(df_coords)):
    lat = float(df_coords['latitude'].iloc[i])
    lon = float(df_coords['longitude'].iloc[i])
    folium.CircleMarker(location=[lat, lon], radius=5, color='red', fill=True, fill_color='red').add_to(m)
for i in range(len(df)):
    lat = float(df['latitude'].iloc[i])
    lon = float(df['longitude'].iloc[i])
    folium.CircleMarker(location=[lat, lon], radius=5, color='green', fill=True, fill_color='green').add_to(m)

# Afficher la carte
m

### V. Analyse et Limites

L'un des problèmes de cette représentation, c'est que les usagers parfois possèdent un point aveugle car ils ne peuvent pas voir les données en temps réel sur une carte. Cela a pour conséquence que l'on retrouve des zones délaissées. On pourrait donc se demander est-ce que il serait judicieux de placer un point à cet emplacement ?

Un autre autre problème que l'on peut relever c'est la prise en compte des pentes sur Brest.

C'est là qu'intervient le solution du _"Toxic waste dump problem"_ (aussi appelé Largest Empty circle Problem) et celle du _"Nearest Neighbour Interpolation"_ qui pourrait être une perspective d'amélioration pour avoir des statistiques plus pertinentes. 

Les avantages de ces solutions :
  * Prise en compte des dénivelés => grâce au _"Nearest Neighbour Interpolation"_
  * Choix de l'emplacement idéal pour placer de nouveau site => grâce au _"Toxic waste dump problem"_